Variables:

1. Monthly returns including dividend distributions

ret

2. Share Price 

prc

3. Common Shares Outstanding 

shrout

4. Share code  

shrcd

5. CRSP permanent company identifier (Permno and or Permco) 

permco 

6. Delisting return 

dlret

7. Date 

8. CRSP value‐weighted index returns

vwretd

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import regex as re
import statsmodels.formula.api as smf

crsp = pd.read_csv('/Users/kailiao/Downloads/crsp.csv', dtype = object)

# change strings or floats to integers
for col in ['PERMNO','PERMCO']:
    crsp[col] = crsp[col].astype(int)

# change returns and prices to floats. If error, coerce to NaN
for col in ['PRC', 'RET', 'vwretd', 'SHROUT'] :
    crsp[col] = pd.to_numeric(crsp[col], errors='coerce')

# deal with negative price
crsp['PRC'] = crsp['PRC'].abs()

crsp['year']  = crsp['date'].str.slice(start=0, stop=4).astype(int)
crsp['month'] = crsp['date'].str.slice(start=4, stop=6).astype(int)
crsp['mrkcap'] = crsp['PRC'] * crsp['SHROUT']

/Users/kailiao/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/kailiao/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/kailiao/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
periods = [1973 + i for i in np.arange(9) * 5]
periods

[1973, 1978, 1983, 1988, 1993, 1998, 2003, 2008, 2013]

In [3]:
pd.options.mode.chained_assignment = None

period= 1973
crsp_1 = crsp[(crsp['year'] < period + 3) & (crsp['year'] >= period)]

def kill_nan_ret(df):
    if df['RET'].isnull().values.any():
        df['mrkcap'] = 0
    return df

crsp_1 = crsp_1.groupby('PERMNO').apply(kill_nan_ret)

# select the largest 400 firms
largest = list(crsp_1.groupby('PERMNO').first().nlargest(n=500, columns=['mrkcap']).reset_index().PERMNO)
crsp_2 = pd.DataFrame()
for firm in largest:
    crsp_temp = crsp_1[crsp_1['PERMNO'] == firm]
    model = smf.ols("RET ~ vwretd", data=crsp_temp)
    result = model.fit()
    crsp_temp['beta'] = result.params.vwretd
    crsp_2 = pd.concat([crsp_2, crsp_temp])

crsp_2 = crsp_2.merge(crsp_2.groupby('PERMNO').first().beta.rank().reset_index(), on='PERMNO')
crsp_2 = crsp_2.rename(columns={"beta_x" : "beta", "beta_y" : "beta_rank"})


In [11]:
# generate portfolios

grids = [i for i in np.arange(10) * 50]
for grid in grids:
    crsp_2.loc[(crsp_2["beta_rank"] > grid)&(crsp_2["beta_rank"] <= grid + 50), 'port'] = np.floor(grid/50) + 1

crsp_2

,PERMNO,date,SHRCD,TICKER,PERMCO,DLRET,PRC,RET,SHROUT,vwretd,year,month,mrkcap,beta,beta_rank,port
0,12490,19730131,11,IBM,20990,NaN,435.500,0.083333,116171.0,-0.027454,1973,1,5.059247e+07,0.814789,124.0,3.0
1,12490,19730228,11,IBM,20990,NaN,431.500,-0.005970,116171.0,-0.044370,1973,2,5.012779e+07,0.814789,124.0,3.0
2,12490,19730330,11,IBM,20990,NaN,431.500,0.000000,116171.0,-0.007857,1973,3,5.012779e+07,0.814789,124.0,3.0
3,12490,19730430,11,IBM,20990,NaN,408.000,-0.054461,116171.0,-0.051769,1973,4,4.739777e+07,0.814789,124.0,3.0
4,12490,19730531,11,IBM,20990,NaN,314.750,-0.032261,145214.0,-0.024508,1973,5,4.570611e+07,0.814789,124.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,25137,19750829,11,CCLR,800,NaN,15.375,-0.075188,11007.0,-0.023558,1975,8,1.692326e+05,0.879222,153.0,4.0
17996,25137,19750930,11,CCLR,800,NaN,14.375,-0.059837,11007.0,-0.038010,1975,9,1.582256e+05,0.879222,153.0,4.0
17997,25137,19751031,11,CCLR,800,NaN,16.375,0.139130,11007.0,0.055857,1975,10,1.802396e+05,0.879222,153.0,4.0
17998,25137,19751128,11,CCLR,800,NaN,16.375,0.000000,11007.0,0.031222,1975,11,1.802396e+05,0.879222,153.0,4.0


Take the 10 beta‐sub‐samples you formed each period based on the firm level beta estimates and form 10 value‐weighted monthly portfolios for each three‐year window. Remember, the weights for each monthly return should be based on the previous month market capitalization. Be sure the weights sum to one each period! You should now have 10 time‐series of monthly beta‐portfolio returns in each period for a total of $10portfolios*36months*8periods=2,880 observation$.

In [20]:
def port_weight(df):
    df['weight'] = df['mrkcap']/df['mrkcap'].sum()
    return df

ports = [i for i in np.arange(10) + 1]
port = 1
crsp_2['weight'] = np.nan
crsp_2 = crsp_2.groupby(['port', 'date']).apply(port_weight)
# weighted return for the stock in a portfolio
crsp_2['portwret'] = crsp_2['RET'] * crsp_2['weight']

,PERMNO,date,SHRCD,TICKER,PERMCO,DLRET,PRC,RET,SHROUT,vwretd,year,month,mrkcap,beta,beta_rank,port,weight,portwret
0,12490,19730131,11,IBM,20990,NaN,435.500,0.083333,116171.0,-0.027454,1973,1,5.059247e+07,0.814789,124.0,3.0,0.331231,0.027602
1,12490,19730228,11,IBM,20990,NaN,431.500,-0.005970,116171.0,-0.044370,1973,2,5.012779e+07,0.814789,124.0,3.0,0.337777,-0.002017
2,12490,19730330,11,IBM,20990,NaN,431.500,0.000000,116171.0,-0.007857,1973,3,5.012779e+07,0.814789,124.0,3.0,0.339351,0.000000
3,12490,19730430,11,IBM,20990,NaN,408.000,-0.054461,116171.0,-0.051769,1973,4,4.739777e+07,0.814789,124.0,3.0,0.333336,-0.018154
4,12490,19730531,11,IBM,20990,NaN,314.750,-0.032261,145214.0,-0.024508,1973,5,4.570611e+07,0.814789,124.0,3.0,0.334070,-0.010777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,25137,19750829,11,CCLR,800,NaN,15.375,-0.075188,11007.0,-0.023558,1975,8,1.692326e+05,0.879222,153.0,4.0,0.002696,-0.000203
17996,25137,19750930,11,CCLR,800,NaN,14.375,-0.059837,11007.0,-0.038010,1975,9,1.582256e+05,0.879222,153.0,4.0,0.002683,-0.000161
17997,25137,19751031,11,CCLR,800,NaN,16.375,0.139130,11007.0,0.055857,1975,10,1.802396e+05,0.879222,153.0,4.0,0.002919,0.000406
17998,25137,19751128,11,CCLR,800,NaN,16.375,0.000000,11007.0,0.031222,1975,11,1.802396e+05,0.879222,153.0,4.0,0.002872,0.000000


In [49]:
portfolios = pd.DataFrame(crsp_2.groupby(['port','date']).portwret.sum()).reset_index()
portfolios.rename(columns={'portwret' : 'portret'})
portfolios = portfolios.merge(crsp_2[['date','vwretd']].drop_duplicates(), on='date')
portfolios

,port,date,portwret,vwretd
0,1.0,19730131,0.005835,-0.027454
1,2.0,19730131,0.006360,-0.027454
2,3.0,19730131,0.014912,-0.027454
3,4.0,19730131,0.004928,-0.027454
4,5.0,19730131,-0.029178,-0.027454
...,...,...,...,...
355,6.0,19751231,-0.007657,-0.010998
356,7.0,19751231,-0.015676,-0.010998
357,8.0,19751231,-0.017735,-0.010998
358,9.0,19751231,-0.013070,-0.010998


In [ ]:
def beta_port(df):
    model = smf.ols("RET ~ vwretd", data=crsp_temp)
    esult = model.fit()
    crsp_temp['beta'] = result.params.vwretd


crsp_temp = crsp_1[crsp_1['PERMNO'] == firm]
model = smf.ols("RET ~ vwretd", data=crsp_temp)
result = model.fit()
crsp_temp['beta'] = result.params.vwretd